# Question 2 (BLP)

We assume the demand model $$u_{ijt} = X_{jt}\beta + \sigma_{B}\nu_{i} + \alpha p_{jt} + \sigma_{I}I_{i}p_{jt} + \xi_{jt} + \epsilon_{ijt}$$ and use BLP to estimate it. The steps are detailed below:

## Step 1: Simulate draws

The logit specification of our model implies that our shares can be written as an integral of fractions of utility components. Specifically, we can adapt the formulas from class to our model: $$s_{jt} = \int \frac{X_{jt}\beta + \sigma_{B}\nu_{i} + \alpha p_{jt} + \sigma_{I}I_{it}p_{jt} + \xi_{jt}}{1+\sum_{m=1}^{J}X_{mt}\beta + \sigma_{B}\nu_{i} + \alpha p_{mt} + \sigma_{I}I_{it}p_{mt} + \xi_{mt}}dP_{I}(I)dP_{\nu}(\nu)$$ 
While this integral cannot be solved analytically, we can simulate random draws of income (uniformly over the income data we have for each product-market) and normal demand shocks to estimate shares conditional on demand parameters:
$$\hat{s}_{jt} = \sum_{r}\frac{X_{jt}\beta + \sigma_{B}\nu_{r} + \alpha p_{jt} + \sigma_{I}I_{rt}p_{jt} + \xi_{jt}}{1+\sum_{m=1}^{J}X_{mt}\beta + \sigma_{B}\nu_{r} + \alpha p_{mt} + \sigma_{I}I_{rt}p_{mt} + \xi_{mt}}$$
We code a function that returns this share while taking demand parameters as input below.

To get closer to the notation used in the lecture notes ($\beta$ is used differently in the problem set,) we will express the deterministic component of utility as $\delta_{jt}$ = $X_{jt}\beta + \alpha p_{jt} + \xi_{jt}$ and the variables that interact with the idiosyncratic components as $\sigma = (\sigma_{B},\sigma_{I})$.

In [1]:
import numpy as np
import pandas as pd
import os

data = pd.read_csv('./cleaned_data/data.csv')
data = data.drop(columns='Unnamed: 0')

demo = pd.read_csv('./PS1_Data/OTCDemographics.csv',sep='\t')
data = pd.merge(data,demo,how='left',left_on=['store','week'],right_on=['store','week'],validate='m:1')
data.head()

,store,week,brand,sales_,count,price_,prom_,cost_,rev,total_cost,...,hhincome11,hhincome12,hhincome13,hhincome14,hhincome15,hhincome16,hhincome17,hhincome18,hhincome19,hhincome20
0,2,1,1,16,14181,3.29,0.0,2.06,52.64,32.96,...,9.629704,12.109250,12.435180,11.096550,9.074136,11.56383,10.475980,11.31761,10.95628,10.73356
1,2,2,1,12,13965,3.27,0.0,2.04,39.24,24.48,...,9.627645,9.719968,11.206970,10.461870,10.513160,11.65730,8.476680,11.32159,10.28266,10.02471
2,2,3,1,6,13538,3.37,0.0,2.15,20.22,12.90,...,10.185430,10.272690,10.556770,8.676463,10.341440,11.64640,9.590904,11.96942,11.36072,11.35747
3,2,4,1,12,13735,3.30,0.0,2.07,39.60,24.84,...,11.982770,8.499218,8.030489,11.080500,8.894314,10.27435,11.675790,12.48191,10.79339,11.42795
4,2,5,1,10,13735,3.34,0.0,2.12,33.40,21.20,...,9.367624,9.109353,10.237140,11.982330,10.211610,10.33920,11.067370,11.89204,10.28755,11.68925


In [2]:
import random
R = 100

# Simulate an individual draw in a single market: each row is a jt-level entry
# Assumes delta is already calculated
# def calc_V(row,sigma_B,sigma_I,nu,hh):
    
#     return row['delta'] + sigma_B*nu + sigma_I*row['hhincome'+str(hh)]*row['price_']


# Simulate draws R=100 times and average out shares
# def sim_shares(orig,sigma_B,sigma_I):

#     # Keep covariates and intermediate variables
#     master = orig[['store','week','brand','sales_','price_','prom_','brand_2','brand_3','brand_4','brand_5','brand_6','brand_7','brand_8','brand_9','brand_10','brand_11','hhincome1','hhincome2','hhincome3','hhincome4','hhincome5','hhincome6','hhincome7','hhincome8','hhincome9','hhincome10','hhincome11','hhincome12','hhincome13','hhincome14','hhincome15','hhincome16','hhincome17','hhincome18','hhincome19','hhincome20','delta','w_old','w_new','s0','count']]
#     # tot is total of all simulated shares
#     master['tot'] = 0

#     # Simulate draws
#     for i in range(R):
#         data_copy = master.copy()
        
#         # Demand shock
#         nu = np.random.normal()
#         # Choose income randomly
#         hh = random.randint(1,20)
        
#         # data_copy['V'] = data_copy.apply(calc_V,axis=1,args=(sigma_B,sigma_I,nu,hh))
#         data_copy['V'] = data_copy['delta'] + sigma_B*nu + sigma_I*data_copy['hhincome'+str(hh)]*data_copy['price_']
        
#         # Use logit to calculate product shares in market
#         data_sum = data_copy.groupby(['store', 'week'],as_index=False)['V'].sum()
#         data_sum.rename(columns={'V':'sum'},inplace=True)
#         data_sum['sum'] = data_sum['sum'] + 1
#         data_copy = pd.merge(data_copy,data_sum,how='left',left_on=['store','week'],right_on=['store','week'],validate='m:1')
#         data_copy['s'] = data_copy['V']/data_copy['sum']
#         data_copy = data_copy[['store','week','brand','s']]

#         # Add this iteration to our total
#         master = pd.merge(master,data_copy,how='left',left_on=['store','week','brand'],right_on=['store','week','brand'],validate='1:1')
#         master['tot'] = master['tot'] + master['s']
#         master = master.drop(columns=['s'])

#     # Average share
#     master['tot'] = master['tot']/R
#     return master
def sim_shares(master,sigma_B,sigma_I):

    # Keep covariates and intermediate variables
    master = master[['store','week','brand','ms_by_store_week','sales_','price_','prom_','brand_2','brand_3','brand_4','brand_5','brand_6','brand_7','brand_8','brand_9','brand_10','brand_11','hhincome1','hhincome2','hhincome3','hhincome4','hhincome5','hhincome6','hhincome7','hhincome8','hhincome9','hhincome10','hhincome11','hhincome12','hhincome13','hhincome14','hhincome15','hhincome16','hhincome17','hhincome18','hhincome19','hhincome20','w_old','w_new','count']]
    
    # tot is total of all simulated shares
    # master['tot'] = 0
    total = pd.Series([0 for _ in range(len(master))])
    
    nus = np.random.standard_normal(R)
    # Simulate draws
    for i in range(R):
        data_copy = master.copy()
        # Demand shock
        nu = nus[i]
        # Choose income randomly
        hh = random.randint(1,20)

        data_copy['V'] = data_copy['w_old']*np.exp(sigma_B*nu + sigma_I*data_copy[f'hhincome{hh}']*data_copy['price_'])
        # Use logit to calculate product shares in market
        data_sum = data_copy.groupby(['store', 'week'],as_index=False)['V'].sum()
        data_sum.rename(columns={'V':'sum'},inplace=True)
        data_sum['sum'] = data_sum['sum'] + 1
        data_copy = pd.merge(data_copy,data_sum,how='left',left_on=['store','week'],right_on=['store','week'],validate='m:1')
        s = data_copy['V']/data_copy['sum']
        total += s

    # Average share
    master['est'] = total/R
    return master

## Step 2: Contraction Mapping

Given $sigma_{B}, sigma_{I}$, we can iterate the contraction mapping in the lecture notes to approximate a value of $\delta_{jt}$ for each product-market pair that results in a share close to the actual shares in the data. The equation we iterate to approximate $\delta_{jt}$ is:
$$\exp(\delta^{i+1}_{jt}) = \exp(\delta^{i}_{jt})\frac{s_{jt}^{0}}{s_{jt}(\delta^{i}_{jt},\beta)}$$

In [3]:
# First, calculate actual shares in data to use in iteration
# def calc_initial_shares(orig):

#     # Sum sales
#     data_sum = orig.groupby(['store', 'week'],as_index=False)['sales_'].sum()
#     data_sum.rename(columns={'sales_':'sum'},inplace=True)

#     # Calculate initial shares
#     orig = pd.merge(orig,data_sum,how='left',left_on=['store','week'],right_on=['store','week'],validate='m:1')
#     orig['s0'] = orig['sales_']/orig['count']
#     orig['share0'] = 1 - (orig['sum']/orig['count'])
#     orig['init'] = np.log(orig['s0']/orig['share0'])
#     orig = orig.drop(columns=['sum','share0'])
#     return orig   


# Calculate delta: deterministic component of jt-level utility
# def calc_delta(orig,sigma_B,sigma_I):
    
#     # Calculate initial shares
#     orig = calc_initial_shares(orig)

#     # Initialize search values and threshold
#     epsilon = 0.1
#     orig['w_old'] = np.exp(orig['s0'])
#     orig['w_new'] = 0
#     count = 0

#     # Iterate contraction mapping until threshold is found
#     while True:
#         orig['delta'] = np.log(orig['w_old'])
#         orig = sim_shares(orig,sigma_B,sigma_I)
#         orig['w_new'] = orig['w_old']*orig['s0']/orig['tot']
#         if (np.log(orig['w_new'])-np.log(orig['w_old'])).abs().mean() < epsilon:
#             break
#         orig['w_old'] = orig['w_new']
#         count += 1

#     return np.log(orig['w_new']).to_numpy()

def calc_delta(orig,sigma_B,sigma_I,delta=None):
    # Initialize search values and threshold
    epsilon = 0.01
    orig['w_old'] = np.exp(delta) if delta is not None else np.exp(orig['ms_by_store_week'])
    orig['w_new'] = 0
    count = 0
    
    # Iterate contraction mapping until threshold is found
    while True:
        if any(orig['w_old'].isnull()):
            print(count)
            raise Exception("NaNs")

        orig = sim_shares(orig,sigma_B,sigma_I)
        orig['w_new'] = orig['w_old']*orig['ms_by_store_week']/orig['est']
        
        if np.average(np.log(orig['ms_by_store_week']/orig['est']).abs()) < epsilon:
            break
        if count > 100:
            print("over_count")
            print(np.average(np.log(orig['ms_by_store_week']/orig['est'])))
            break
        orig['w_old'] = orig['w_new']
        count += 1

    return np.log(orig['w_new']).to_numpy()

# Calculate xi: our jt-level residual
# Two parts: iterate contraction mapping, then subtract out linear terms given beta
def calc_xi(orig,sigma_B,sigma_I,beta):

    # Calculate delta
    orig['delta'] = calc_delta(orig,sigma_B,sigma_I)    

    # Calculate xi: take linear component out of delta
    orig['xi'] = orig['delta'] - orig[['price_','brand_2','brand_3','brand_4','brand_5','brand_6','brand_7','brand_8','brand_9','brand_10','brand_11']].dot(beta)
    return orig['xi'].to_numpy()


Below I try to run the above code step by step. Keeping the mess so that you can see the (lack of) convergence: the outputted numbers are the difference between successive values of $\delta_{jt}$, averaged over all jt-pairs. It looks like there's initial convergence, but eventually it starts to explode...

In [5]:
beta = np.zeros(11)
beta[0] = 1
data = calc_initial_shares(data)
epsilon = 0.1
data['w_old'] = np.exp(data['s0'])
data['w_new'] = 0

data['delta'] = np.log(data['w_old'])
data = sim_shares(data,1,1)
data['w_new'] = data['w_old']*data['s0']/data['tot']
print((np.log(data['w_new'])-np.log(data['w_old'])).abs().mean())
data['w_old'] = data['w_new']

/var/folders/l2/qgb9rd795r92jzsy94cy01080000gn/T/ipykernel_36814/998985169.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  master['tot'] = 0


5.489137223491835


In [4]:
while True:
    data['delta'] = np.log(data['w_old'])
    data = sim_shares(data,0,1)
    data['w_new'] = data['w_old']*data['s0']/data['tot']
    print((np.log(data['w_new'])-np.log(data['w_old'])).abs().mean())
    print((data['s0']/data['tot']).mean())
    if (np.log(data['w_new'])-np.log(data['w_old'])).abs().mean() < 0.1:
            break
    data['w_old'] = data['w_new']

KeyError: 'w_old'

In [169]:
data.head()

,store,week,brand,sales_,price_,prom_,brand_2,brand_3,brand_4,brand_5,...,hhincome17,hhincome18,hhincome19,hhincome20,delta,w_old,w_new,s0,count,tot
0,2,1,1,16,3.29,0.0,0,0,0,0,...,10.475980,11.31761,10.95628,10.73356,-8.160519,0.000286,0.000286,0.001128,14181,0.066600
1,2,2,1,12,3.27,0.0,0,0,0,0,...,8.476680,11.32159,10.28266,10.02471,-8.731163,0.000161,0.000161,0.000859,13965,0.067198
2,2,3,1,6,3.37,0.0,0,0,0,0,...,9.590904,11.96942,11.36072,11.35747,-10.129091,0.000040,0.000040,0.000443,13538,0.069380
3,2,4,1,12,3.30,0.0,0,0,0,0,...,11.675790,12.48191,10.79339,11.42795,-8.753028,0.000158,0.000158,0.000874,13735,0.069164
4,2,5,1,10,3.34,0.0,0,0,0,0,...,11.067370,11.89204,10.28755,11.68925,-9.096025,0.000112,0.000112,0.000728,13735,0.068305


## Step 3: Define GMM objective function

We now use our instruments to define an objective function which is to be minimized to find our optimal paramters $\beta$, $\sigma_{B}$, and $\sigma_{I}$. Using the formula found in Nevo's RA guide, we can express $\beta$ as a function of $(\sigma_{B},\sigma_{I})$: 
$$\beta = (X^{T}ZWZ^{T}X)^{-1}X^{T}ZWZ^{T}\delta(\sigma_{B},\sigma_{I})$$  
With $\beta$ in hand, we can now calculate $\xi(\sigma_{B},\sigma_{I},\beta)$ and thus our entire objective function:
$$\xi^{T}ZWZ^{T}\xi$$

In [11]:
instr = pd.read_csv('./PS1_Data/OTCDataInstruments.csv',sep='\t')
instr = instr.drop(columns=['store','week','brand','avoutprice'])
Z = instr.to_numpy()
W = np.linalg.inv(np.matmul(np.transpose(Z),Z))
X = data[['price_','brand_2','brand_3','brand_4','brand_5','brand_6','brand_7','brand_8','brand_9','brand_10','brand_11']].to_numpy()

def gmm_obj(sigma):
    sigma_B = sigma[0]
    sigma_I = sigma[1]
    print(sigma)
    proj = np.linalg.inv(np.matmul(np.transpose(X),np.matmul(Z,np.matmul(W,np.matmul(np.transpose(Z),X)))))
    vect = np.matmul(np.transpose(X),np.matmul(Z,np.matmul(W,np.matmul(np.transpose(Z),calc_delta(data,sigma_B,sigma_I)))))
    beta = np.matmul(proj,vect)
    xi = calc_xi(data,sigma_B,sigma_I,beta)
    ans = np.matmul(np.transpose(xi),np.matmul(Z,np.matmul(W,np.matmul(np.transpose(Z),xi))))
    first_comp = np.array([1,0])
    second_comp = np.array([0,1])
    return [np.matmul(first_comp,ans),np.matmul(second_comp,ans)]

## Step 4: Nonlinear search over parameters

Now that we've defined a loss function to minimize, we look for parameters $\sigma_{B}, \sigma_{I}$ that minimize it. We use scipy's fsolve, which relies on MINPACK's hybrid algorithm, for nonlinear optimization.

In [12]:
from scipy.optimize import fsolve

pd.set_option('mode.chained_assignment', None)
(sigma_B,sigma_I) = fsolve(gmm_obj,[1,1])
proj = np.linalg.inv(np.matmul(np.transpose(X),np.matmul(Z,np.matmul(W,np.matmul(np.transpose(Z),X)))))
vect = np.matmul(np.transpose(X),np.matmul(Z,np.matmul(W,np.matmul(np.transpose(Z),calc_delta(data,sigma_B,sigma_I)))))
beta = np.matmul(proj,vect)
alpha = beta[0]

[1 1]
over_count
9.451880841204814e-05
over_count
0.05007320425449573


ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

## Elasticity calculation

Unlike the logit specification, elasticities under BLP need to be simulated. We will simulate: 
$$ e_{jjt} = -\frac{p_{jt}}{s_{jt}}\int (\alpha + \sigma_{I}I_{i})Pr_{ijt}(1-Pr_{ijt})dP_{D}(D)dP_{\nu}(\nu) $$ $$ e_{jkt} = \frac{p_{kt}}{s_{jt}} \int (\alpha + \sigma_{I}I_{i})Pr_{ijt}Pr_{ikt}dP_{D}(D)dP_{\nu}(\nu)$$
where $Pr_{ijt}$ is the probability of $i$ choosing $j$, simulated using the procedure in step 1.

In [ ]:
# Calculate delta for each jt-pair
data['delta'] = calc_delta(data,sigma_B,sigma_I)

In [ ]:
def calc_own_e(orig,alpha,sigma_B,sigma_I):

    # e_own is total of all simulated share-price derivatives
    orig['e_own'] = 0

    # Simulate draws
    for i in range(R):
        data_copy = master.copy()
        
        # Demand shock
        nu = np.random.normal()
        # Choose income randomly
        hh = random.randint(1,20)
        
        data_copy['V'] = data_copy.apply(calc_V,axis=1,args=(sigma_B,sigma_I,nu,hh))

        # Use logit to calculate product shares in market
        data_sum = data_copy.groupby(['store', 'week'],as_index=False)['V'].sum()
        data_sum.rename(columns={'V':'sum'},inplace=True)
        data_sum['sum'] = data_sum['sum'] + 1
        data_copy = pd.merge(data_copy,data_sum,how='left',left_on=['store','week'],right_on=['store','week'],validate='m:1')
        data_copy['s'] = data_copy['V']/data_copy['sum']
        data_copy['dsdp'] = data_copy['s']*(1-data_copy['s'])*(alpha + sigma_I*data_copy['hhincome'+str(hh)])
        data_copy = data_copy[['store','week','brand','dsdp']]

        # Add this iteration to our total
        orig = pd.merge(orig,data_copy,how='left',left_on=['store','week','brand'],right_on=['store','week','brand'],validate='1:1')
        orig['e_own'] = orig['e_own'] + orig['dsdp']
        orig = orig.drop(columns=['dsdp'])

    # Calculate elasticity
    orig['e_own'] = -1*orig['price_']*orig['e_own']/R
    orig['e_own'] = orig['e_own']*orig['sales_']/orig['count']
    return orig


def calc_cross_e(orig,alpha,sigma_I,k):

    orig['e_'+str(k)] = 0
    
    for i in range(R):
        data_copy = orig.copy()
        
        # Demand shock
        nu = np.random.normal()
        # Choose income randomly
        hh = random.randint(1,20)

        # Calculate utility
        data_copy['V'] = data_copy.apply(calc_V,axis=1,args=(sigma_B,sigma_I,nu,hh))

        # Use logit to calculate product shares in market
        data_sum = data_copy.groupby(['store', 'week'],as_index=False)['V'].sum()
        data_sum.rename(columns={'V':'sum'},inplace=True)
        data_sum['sum'] = data_sum['sum'] + 1
        data_copy = pd.merge(data_copy,data_sum,how='left',left_on=['store','week'],right_on=['store','week'],validate='m:1')
        data_copy['s'] = data_copy['V']/data_copy['sum']

        data_k = orig[orig['brand']==k]
        data_k = data_k.rename(columns={'s':'s_k'})
        data_k = data_k[['store','week','brand','s_k']]
        data_copy = pd.merge(data_copy,data_k,how='left',left_on=['store','week'],right_on=['store','week'],validate='m:1')
        
        data_copy['dsdp'] = data_copy['s']*(data_copy['s_k'])*(alpha + sigma_I*data_copy['hhincome'+str(hh)])
        data_copy = data_copy[['store','week','brand','dsdp']]

        # Add this iteration to our total
        orig = pd.merge(orig,data_copy,how='left',left_on=['store','week','brand'],right_on=['store','week','brand'],validate='1:1')
        orig['e_'+str(k)] = orig['e_'+str(k)] + orig['dsdp']
        orig = orig.drop(columns=['dsdp'])

    data_k = orig[orig['brand']==k]
    data_k = data_k.rename(columns={'price_':'price_k'})
    data_k = data_k[['store','week','price_k']]
    orig = pd.merge(orig,data_k,how='left',left_on=['store','week'],right_on=['store','week'],validate='m:1')
    orig['e_'+str(k)] = orig['e_'+str(k)]*orig['price_k']*orig['sales_']/(orig['count']*R)
    
    return orig

In [ ]:
# 2b: Elasticities for store 9, week 10
data = calc_own_e(data,alpha,sigma_B,sigma_I)
for i in range(1,12):
    data = calc_cross_e(data,alpha,sigma_I,i)

data_ans = data[((data['week'] == 10) & (data['store'] == 9))]
data_ans = data_ans[['brand','e_own','e_1','e_2','e_3','e_4','e_5','e_6','e_7','e_8','e_9','e_10','e_11']]
for i in range(1,12):
    data[data['brand']==i]['e_'+str(i)] = data['e_own']